In [23]:
# Importing dependencies 
import pandas as pd
import numpy as np
import scipy.stats as st
import os

# Importing time traveled to work (county and state) csvs and concatonating into a sigle dataframe
path1 = 'Resources/Travel Time to Work'
files1 = os.listdir(path1)
travel_df = pd.DataFrame()
for file in files1:
    temp1 = pd.read_csv(path1 + '\\' + file, skiprows=[1])
    temp1 = temp1[['B08303_001E', 'B08303_002E', 'B08303_003E', 'B08303_004E', 'B08303_005E', 'B08303_006E', 'B08303_007E', \
        'B08303_008E', 'B08303_009E', 'B08303_010E', 'B08303_011E', 'B08303_012E', 'B08303_013E', 'NAME']]
    #Renaming columns 
    temp1.columns = ['Total Estimate', 'Estimate Less than 5 min', 'Estimate 5 to 9 min', 'Estimate 10 to 14 min', \
        'Estimate 15 to 19 min', 'Estimate 20 to 24 min', 'Estimate 25 to 29 min', 'Estimate 30 to 34 min', 'Estimate 35 to 39 min', \
        'Estimate 40 to 44 min', 'Estimate 45 to 59 min', 'Estimate 60 to 89 min', 'Estimate 90 or more min', 'State']
    # Adding a column for respective years
    temp1['Year'] = file.split(' ')[0]
    # reordering columns
    temp1 = temp1[['Year', 'State', 'Total Estimate', 'Estimate Less than 5 min', 'Estimate 5 to 9 min', 'Estimate 10 to 14 min', \
        'Estimate 15 to 19 min', 'Estimate 20 to 24 min', 'Estimate 25 to 29 min', 'Estimate 30 to 34 min', 'Estimate 35 to 39 min', \
        'Estimate 40 to 44 min', 'Estimate 45 to 59 min', 'Estimate 60 to 89 min', 'Estimate 90 or more min']]
    # Removing Puerto Rico
    temp1 = temp1[temp1['State'].str.contains('Puerto Rico')==False]
    # Appending each new temporary dataframe into one
    travel_df = travel_df.append(temp1)

# Importing time traveled to work (county and state) csvs and concatonating into a sigle dataframe
path2 = 'Resources/Travel Time County and State Level'
files2 = os.listdir(path2)
cs_travel_df = pd.DataFrame()
for file in files2:
    temp = pd.read_csv(path2 + '\\' + file, skiprows=[1])
    temp = temp[['B08131_001E', 'B08131_002E', 'B08131_003E', 'B08131_004E', 'B08131_005E', 'NAME']]
    # Renaming columns 
    temp.columns = ['Estimate Aggregate Travel Time (min)', 'Worked in State of Residence Estimate Aggregate (min)', \
        'Worked in County of Residence Estimate Aggregate (min)', 'Worked Outside County of Residence Estimate Aggregate (min)', \
        'Worked Outside State of Residence Estimate Aggregate (min)', 'State']
    # Adding a column for respective years
    temp['Year'] = file.split(' ')[0]
    # Reordering columns
    temp = temp[['Year', 'State', 'Estimate Aggregate Travel Time (min)', 'Worked in State of Residence Estimate Aggregate (min)', \
        'Worked in County of Residence Estimate Aggregate (min)', 'Worked Outside County of Residence Estimate Aggregate (min)', \
        'Worked Outside State of Residence Estimate Aggregate (min)']]
    # Removing '-' values 
    temp['Worked Outside County of Residence Estimate Aggregate (min)'] = pd.to_numeric(temp['Worked Outside County of Residence Estimate Aggregate (min)'], \
                                                                                       errors='coerce')
    temp = temp[temp['Worked Outside County of Residence Estimate Aggregate (min)'] != '-']
    # Removing Puerto Rico
    temp = temp[temp['State'].str.contains('Puerto Rico')==False]
    # Removing "-" values
    temp['Worked Outside County of Residence Estimate Aggregate (min)'] = pd.to_numeric(temp['Worked Outside County of Residence Estimate Aggregate (min)'], errors='coerce')
    # Appending each new temporary dataframe into one
    cs_travel_df = cs_travel_df.append(temp)

# Importing HPI csv and making into a dataframe
hpi_quarterly = pd.read_csv('Resources/HPI/HPI Quarterly State Level.csv')
# Removing Warning column
hpi_quarterly = hpi_quarterly[['state', 'yr', 'qtr', 'index_nsa', 'index_sa']]
# Renaming columns
hpi_quarterly.columns = ['State', 'Year', 'Quarter', 'NSA Index', 'SA Index']
# Gropuby to get mean of SA and NSA index for each state and year
hpi_yearly = hpi_quarterly.groupby(['State', 'Year'], as_index=False).agg({'NSA Index': 'mean', 'SA Index': 'mean'})
# Renaming columns again 
hpi_yearly.columns = ['State', 'Year', 'Yearly Average Index (NSA)', 'Yearly Average Index (SA)']
# Only keeping years 2010-2019
hpi_yearly = hpi_yearly.loc[(hpi_yearly['Year'].astype(int) >= 2010) & (hpi_yearly['Year'].astype(int) <= 2019)]

In [24]:
us_state_to_abbrev = {"Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", "California": "CA", "Colorado": "CO", \
    "Connecticut": "CT", "Delaware": "DE", "Florida": "FL", "Georgia": "GA", "Hawaii": "HI", "Idaho": "ID", "Illinois": "IL", \
    "Indiana": "IN", "Iowa": "IA", "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Maryland": "MD", \
    "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN", "Mississippi": "MS", "Missouri": "MO", "Montana": "MT", "Nebraska": "NE", \
    "Nevada": "NV", "New Hampshire": "NH", "New Jersey": "NJ", "New Mexico": "NM", "New York": "NY", "North Carolina": "NC", "North Dakota": "ND", \
    "Ohio": "OH", "Oklahoma": "OK", "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC", "South Dakota": "SD", \
    "Tennessee": "TN", "Texas": "TX", "Utah": "UT", "Vermont": "VT", "Virginia": "VA", "Washington": "WA", "West Virginia": "WV", \
    "Wisconsin": "WI", "Wyoming": "WY", "District of Columbia": "DC"
}

In [25]:
# Changing state names to abbreviations
travel_df = travel_df.replace({'State': us_state_to_abbrev})
cs_travel_df = cs_travel_df.replace({'State': us_state_to_abbrev})

# Merging HPI and travel time dataframes
travels = pd.merge(travel_df, cs_travel_df, on=['State', 'Year'], how='left')
# Changing HPI year column back into strings to maintain compatibility 
hpi_yearly['Year'] = hpi_yearly['Year'].astype(str)
travel_hpi = pd.merge(travels, hpi_yearly, on=['State', 'Year'], how='left')

# Saving merged dataframe in Resources folder
#travel_hpi.to_csv('Resources/2010 to 2019 HPI and Travel Time to Work.csv')

In [26]:
#creating % of total columns for travel time estimates and county and state level travel estimates

# % of total travel time estimates
travel_hpi['% Travel Less than 5 min'] = travel_hpi['Estimate Less than 5 min']/travel_hpi['Total Estimate']
travel_hpi['% Travel 5 to 9 min'] = travel_hpi['Estimate 5 to 9 min']/travel_hpi['Total Estimate']
travel_hpi['% Travel 10 to 14 min'] = travel_hpi['Estimate 10 to 14 min']/travel_hpi['Total Estimate']
travel_hpi['% Travel 15 to 19 min'] = travel_hpi['Estimate 15 to 19 min']/travel_hpi['Total Estimate']
travel_hpi['% Travel 20 to 24 min'] = travel_hpi['Estimate 20 to 24 min']/travel_hpi['Total Estimate']
travel_hpi['% Travel 25 to 29 min'] = travel_hpi['Estimate 25 to 29 min']/travel_hpi['Total Estimate']
travel_hpi['% Travel 30 to 34 min'] = travel_hpi['Estimate 30 to 34 min']/travel_hpi['Total Estimate']
travel_hpi['% Travel 35 to 39 min'] = travel_hpi['Estimate 35 to 39 min']/travel_hpi['Total Estimate']
travel_hpi['% Travel 40 to 44 min'] = travel_hpi['Estimate 40 to 44 min']/travel_hpi['Total Estimate']
travel_hpi['% Travel 45 to 59 min'] = travel_hpi['Estimate 45 to 59 min']/travel_hpi['Total Estimate']
travel_hpi['% Travel 60 to 89 min'] = travel_hpi['Estimate 60 to 89 min']/travel_hpi['Total Estimate']
travel_hpi['% Travel More than 90 min'] = travel_hpi['Estimate 90 or more min']/travel_hpi['Total Estimate']
           
# % of workers in county, out of county, or out of state          
travel_hpi['% Working in County of Residence'] = travel_hpi['Worked in County of Residence Estimate Aggregate (min)']/travel_hpi['Estimate Aggregate Travel Time (min)']
travel_hpi['% Working Outside County of Residence'] = travel_hpi['Worked Outside County of Residence Estimate Aggregate (min)']/travel_hpi['Estimate Aggregate Travel Time (min)']
travel_hpi['% Working in State of Residence'] = travel_hpi['Worked in State of Residence Estimate Aggregate (min)']/travel_hpi['Estimate Aggregate Travel Time (min)']
travel_hpi['% Working Outside State of Residence'] = travel_hpi['Worked Outside State of Residence Estimate Aggregate (min)']/travel_hpi['Estimate Aggregate Travel Time (min)']

In [27]:
travel_hpi.to_csv('Resources/2010 to 2019 HPI and Travel Time to Work.csv', index=False)

In [28]:
travel_hpi.head()

,Year,State,Total Estimate,Estimate Less than 5 min,Estimate 5 to 9 min,Estimate 10 to 14 min,Estimate 15 to 19 min,Estimate 20 to 24 min,Estimate 25 to 29 min,Estimate 30 to 34 min,...,% Travel 30 to 34 min,% Travel 35 to 39 min,% Travel 40 to 44 min,% Travel 45 to 59 min,% Travel 60 to 89 min,% Travel More than 90 min,% Working in County of Residence,% Working Outside County of Residence,% Working in State of Residence,% Working Outside State of Residence
0,2010,AL,1895987,59051,189676,274724,330572,301885,117283,264594,...,0.139555,0.028287,0.029620,0.071576,0.040230,0.019213,0.568210,0.360792,0.929003,0.070997
1,2010,AK,324619,30199,56593,58333,59252,44082,16062,24726,...,0.076169,0.016170,0.012430,0.037672,0.025242,0.017451,0.815196,0.179669,0.994864,0.005135
2,2010,AZ,2470950,66596,243712,346279,385741,376165,166696,390622,...,0.158086,0.028750,0.042460,0.072769,0.040357,0.016048,0.888187,0.085778,0.973965,0.026035
3,2010,AR,1189709,60584,171728,203607,207225,163084,73975,136006,...,0.114319,0.025547,0.023542,0.050491,0.029665,0.016589,0.567898,0.366222,0.934120,0.065880
4,2010,CA,15097170,346580,1331209,2102009,2350056,2255417,851250,2224075,...,0.147317,0.025530,0.039092,0.078736,0.067547,0.029972,0.703514,0.289409,0.992924,0.007076


# HPI Summary

In [29]:
df_2010_2019 = travel_hpi.loc[(travel_hpi['Year'] == '2010') | (travel_hpi['Year'] == '2019')]
state_groupby = df_2010_2019.groupby('State')
# Calculating change in HPI (both NSA and SA) over time (2010 to 2019) for each state
states = df_2010_2019['State'].unique()

change_nsa = state_groupby['Yearly Average Index (NSA)'].diff().dropna() 
change_sa = state_groupby['Yearly Average Index (SA)'].diff().dropna()
pct_change_nsa = (state_groupby['Yearly Average Index (NSA)'].pct_change().dropna())*100
pct_change_sa = (state_groupby['Yearly Average Index (SA)'].pct_change().dropna())*100
hpi_change = pd.DataFrame({'State': states, 'Change in NSA Index': change_nsa, '% Change in NSA Index': pct_change_nsa, \
    'Change in SA Index': change_sa, '% Change in SA Index': pct_change_sa})
hpi_change = hpi_change.reset_index(drop=True)
hpi_change.style.format({'% Change in NSA Index': '{:,.2f}%', '% Change in SA Index': '{:,.2f}%'})



,State,Change in NSA Index,% Change in NSA Index,Change in SA Index,% Change in SA Index
0,AL,48.627500,26.90%,48.467500,26.91%
1,AK,45.180000,20.55%,45.012500,20.57%
2,AZ,151.690000,82.82%,151.385000,82.83%
3,AR,46.727500,26.21%,46.552500,26.14%
4,CA,124.192500,75.37%,123.555000,75.35%
5,CO,216.760000,84.16%,215.672500,84.15%
6,CT,7.127500,4.24%,7.125000,4.25%
7,DE,24.967500,13.36%,25.005000,13.43%
8,DC,232.167500,73.43%,230.285000,73.41%
9,FL,139.995000,78.74%,139.717500,78.74%


In [30]:
# Min and Max percentages - HPI
max_nsa = hpi_change['% Change in NSA Index'].max()
min_nsa = hpi_change['% Change in NSA Index'].min()
max_sa = hpi_change['% Change in SA Index'].max()
min_sa = hpi_change['% Change in SA Index'].min()
# States corresponding to min and max
state_max_nsa = hpi_change[hpi_change['% Change in NSA Index'] == max_nsa]['State'].item()
state_min_nsa = hpi_change[hpi_change['% Change in NSA Index'] == min_nsa]['State'].item() 
state_max_sa = hpi_change[hpi_change['% Change in SA Index'] == max_sa]['State'].item()
state_min_sa = hpi_change[hpi_change['% Change in SA Index'] == min_sa]['State'].item()

print(f'Highest % change in HPI (NSA) from 2010 to 2019: {round(max_nsa,2)}%, {state_max_nsa}')
print(f'Lowest % change in HPI (NSA) from 2010 to 2019: {round(min_nsa,2)}%, {state_min_nsa}')
print(f'Highest % change in HPI (SA) from 2010 to 2019: {round(max_sa,2)}%, {state_max_sa}')
print(f'Lowest % change in HPI (SA) from 2010 to 2019: {round(min_sa,2)}%, {state_min_sa}')

hpi_change.to_csv('Resources/2010 to 2019 HPI Change.csv', index=False)

Highest % change in HPI (NSA) from 2010 to 2019: 110.02%, NV
Lowest % change in HPI (NSA) from 2010 to 2019: 4.24%, CT
Highest % change in HPI (SA) from 2010 to 2019: 110.05%, NV
Lowest % change in HPI (SA) from 2010 to 2019: 4.25%, CT


# Time Traveled to Work Summary

In [31]:
# Calculating change in travel time over time (2010 to 2019) for each state
change_less5 = state_groupby['% Travel Less than 5 min'].pct_change().dropna() * 100 
change_5to9 = state_groupby['% Travel 5 to 9 min'].pct_change().dropna() * 100 
change_10to14 = state_groupby['% Travel 10 to 14 min'].pct_change().dropna() * 100
change_15to19 = state_groupby['% Travel 15 to 19 min'].pct_change().dropna() * 100
change_20to24 = state_groupby['% Travel 20 to 24 min'].pct_change().dropna() * 100
change_25to29 = state_groupby['% Travel 25 to 29 min'].pct_change().dropna() * 100
change_30to34 = state_groupby['% Travel 30 to 34 min'].pct_change().dropna() * 100
change_35to39 = state_groupby['% Travel 35 to 39 min'].pct_change().dropna() * 100
change_40to44 = state_groupby['% Travel 40 to 44 min'].pct_change().dropna() * 100
change_45to59 = state_groupby['% Travel 45 to 59 min'].pct_change().dropna() * 100
change_60to89 = state_groupby['% Travel 60 to 89 min'].pct_change().dropna() * 100
change_90more = state_groupby['% Travel More than 90 min'].pct_change().dropna() * 100

travel_time_change = pd.DataFrame({'State': states, '% Change <5 min': change_less5, '% Change 5 to 9 min': change_5to9, \
    '% Change 10 to 14 min': change_10to14, '% Change 15 to 19 min': change_15to19, '% Change 20 to 24 min': change_20to24, \
    '% Change 25 to 29 min': change_25to29, '% Change 30 to 34 min': change_30to34, '% Change 35 to 39 min': change_35to39, \
    '% Change 40 to 44 min': change_40to44, '% Change 45 to 59 min': change_45to59, '% Change 60 to 89 min': change_60to89, \
    '% Change > 89 min': change_90more})
travel_time_change = travel_time_change.reset_index(drop=True)
travel_time_change.style.format({'% Change <5 min': '{:,.2f}%', '% Change 5 to 9 min': '{:,.2f}%', \
    '% Change 10 to 14 min': '{:,.2f}%', '% Change 15 to 19 min': '{:,.2f}%', '% Change 20 to 24 min': '{:,.2f}%', \
    '% Change 25 to 29 min': '{:,.2f}%', '% Change 30 to 34 min': '{:,.2f}%', '% Change 35 to 39 min': '{:,.2f}%', \
    '% Change 40 to 44 min': '{:,.2f}%', '% Change 45 to 59 min': '{:,.2f}%', '% Change 60 to 89 min': '{:,.2f}%', \
    '% Change > 89 min': '{:,.2f}%'})

travel_time_change.to_csv('Resources/2010 to 2019 Travel Time to Change.csv', index=False)

In [33]:
# Creating 5 time groups for travel times
time_groups = travel_hpi

# less than 20, 20-39, 40-59, 60-89, 90+
time_groups['% Travel Less than 20 min'] = (travel_hpi['Estimate Less than 5 min']+ travel_hpi['Estimate 5 to 9 min']\
                                            +travel_hpi['Estimate 10 to 14 min']+travel_hpi['Estimate 15 to 19 min'])/travel_hpi['Total Estimate']
time_groups['% Travel 20 to 39 min'] = (travel_hpi['Estimate 20 to 24 min']+travel_hpi['Estimate 25 to 29 min']\
                                        +travel_hpi['Estimate 30 to 34 min']+ travel_hpi['Estimate 35 to 39 min'])/travel_hpi['Total Estimate']
time_groups['% Travel 40 to 59 min'] = (travel_hpi['Estimate 40 to 44 min']+travel_hpi['Estimate 45 to 59 min'])/travel_hpi['Total Estimate']
time_groups['% Travel 60 to 89 min'] = (travel_hpi['Estimate 60 to 89 min']+travel_hpi['Estimate 45 to 59 min'])/travel_hpi['Total Estimate']
time_groups['% Travel More than 90 min'] = travel_hpi['Estimate 90 or more min']/travel_hpi['Total Estimate']


# Calculating change in travel time over time (2010 to 2019) for each state

df_2010_2019_time = time_groups.loc[(travel_hpi['Year'] == '2010') | (time_groups['Year'] == '2019')]
state_groupby_time = df_2010_2019_time.groupby('State')
# Calculating change in HPI (both NSA and SA) over time (2010 to 2019) for each state
states = df_2010_2019_time['State'].unique()

change_less20 = state_groupby_time['% Travel Less than 20 min'].pct_change().dropna() * 100 
change_20to39 = state_groupby_time['% Travel 20 to 39 min'].pct_change().dropna() * 100 
change_40to59 = state_groupby_time['% Travel 10 to 14 min'].pct_change().dropna() * 100
change_60to89 = state_groupby_time['% Travel 15 to 19 min'].pct_change().dropna() * 100
change_more90 = state_groupby_time['% Travel More than 90 min'].pct_change().dropna() * 100

time_group_change = pd.DataFrame({'State': states, \
    '% Change Travel Less than 20 min': change_less20,\
    '% Change Travel 20 to 39 min': change_20to39, \
    '% Change Travel 40 to 59 min': change_40to59,\
    '% Change Travel 60 to 89 min': change_60to89,\
    '% Change Travel More than 90 min': change_more90})

time_group_change = time_group_change.reset_index(drop=True)

time_group_change.style.format({'% Change Travel Less than 20 min': '{:,.2f}%'.format, '% Change Travel 20 to 39 min': '{:,.2f}%'.format,\
'% Change Travel 40 to 59 min': '{:,.2f}%'.format, '% Change Travel 60 to 89 min': '{:,.2f}%'.format, '% Change Travel More than 90 min': '{:,.2f}%'.format})

,State,% Change Travel Less than 20 min,% Change Travel 20 to 39 min,% Change Travel 40 to 59 min,% Change Travel 60 to 89 min,% Change Travel More than 90 min
0,AL,-4.39%,-0.83%,-4.23%,-2.33%,14.07%
1,AK,0.15%,-1.78%,2.22%,2.51%,36.89%
2,AZ,-8.33%,-0.18%,-12.00%,-6.56%,42.76%
3,AR,-1.82%,-1.50%,3.04%,-1.49%,10.46%
4,CA,-14.52%,0.07%,-15.83%,-8.55%,54.30%
5,CO,-12.30%,4.89%,-11.35%,-3.62%,17.85%
6,CT,-6.70%,-0.55%,-3.64%,-4.96%,45.22%
7,DE,-8.14%,1.92%,-1.25%,-10.07%,41.96%
8,DC,-13.81%,-5.74%,-12.83%,-14.28%,-18.76%
9,FL,-13.72%,0.21%,-16.25%,-9.59%,35.33%


In [34]:
# Min and Max percentages - time traveled groups
max_less20 = time_group_change['% Change Travel Less than 20 min'].max()
min_less20 = time_group_change['% Change Travel Less than 20 min'].min()
max_20to39 = time_group_change['% Change Travel 20 to 39 min'].max()
min_20to39 = time_group_change['% Change Travel 20 to 39 min'].min()
max_40to59 = time_group_change['% Change Travel 40 to 59 min'].max()
min_40to59 = time_group_change['% Change Travel 40 to 59 min'].min()
max_60to89 = time_group_change['% Change Travel 60 to 89 min'].max()
min_60to89 = time_group_change['% Change Travel 60 to 89 min'].min()
max_more90 = time_group_change['% Change Travel More than 90 min'].max()
min_more90 = time_group_change['% Change Travel More than 90 min'].min()

# States corresponding to min and max
state_max_less20 = time_group_change[time_group_change['% Change Travel Less than 20 min'] == max_less20]['State'].item()
state_min_less20 = time_group_change[time_group_change['% Change Travel Less than 20 min'] == min_less20]['State'].item() 
state_max_20to39 = time_group_change[time_group_change['% Change Travel 20 to 39 min'] == max_20to39]['State'].item()
state_min_20to39 = time_group_change[time_group_change['% Change Travel 20 to 39 min'] == min_20to39]['State'].item()
state_max_40to59 = time_group_change[time_group_change['% Change Travel 40 to 59 min'] == max_40to59]['State'].item()
state_min_40to59 = time_group_change[time_group_change['% Change Travel 40 to 59 min'] == min_40to59]['State'].item()
state_max_60to89 = time_group_change[time_group_change['% Change Travel 60 to 89 min'] == max_60to89]['State'].item()
state_min_60to89 = time_group_change[time_group_change['% Change Travel 60 to 89 min'] == min_60to89]['State'].item()
state_max_more90 = time_group_change[time_group_change['% Change Travel More than 90 min'] == max_more90]['State'].item()
state_min_more90 = time_group_change[time_group_change['% Change Travel More than 90 min'] == min_more90]['State'].item()

print(f'Largest % change increase travel less than 20 min from 2010 to 2019: {round(max_less20,2)}%, {state_max_less20}')
print(f'Largest % change decrease travel less than 20 min from 2010 to 2019: {round(min_less20,2)}%, {state_min_less20}')
print(f'Largest % change increase travel 20 to 39 min from 2010 to 2019: {round(max_20to39,2)}%, {state_max_20to39}')
print(f'Largest % change decrease travel 20 to 39 min from 2010 to 2019: {round(min_20to39,2)}%, {state_min_20to39}')
print(f'Largest % change increase travel 40 to 59 min from 2010 to 2019: {round(max_40to59,2)}%, {state_max_40to59}')
print(f'Largest % change decrease travel 40 to 59 min from 2010 to 2019: {round(min_40to59,2)}%, {state_min_40to59}')
print(f'Largest % change increase travel 60 to 89 min from 2010 to 2019: {round(max_60to89,2)}%, {state_max_60to89}')
print(f'Largest % change decrease travel 60 to 89 min from 2010 to 2019: {round(min_60to89,2)}%, {state_min_60to89}')
print(f'Largest % change increase travel more than 90 min from 2010 to 2019: {round(max_more90,2)}%, {state_max_more90}')
print(f'Largest % change decrease travel more than 90 min from 2010 to 2019: {round(min_more90,2)}%, {state_min_more90}')

Largest % change increase travel less than 20 min from 2010 to 2019: 0.15%, AK
Largest % change decrease travel less than 20 min from 2010 to 2019: -16.44%, RI
Largest % change increase travel 20 to 39 min from 2010 to 2019: 18.15%, ND
Largest % change decrease travel 20 to 39 min from 2010 to 2019: -6.43%, HI
Largest % change increase travel 40 to 59 min from 2010 to 2019: 3.04%, AR
Largest % change decrease travel 40 to 59 min from 2010 to 2019: -23.56%, RI
Largest % change increase travel 60 to 89 min from 2010 to 2019: 21.33%, WY
Largest % change decrease travel 60 to 89 min from 2010 to 2019: -14.28%, DC
Largest % change increase travel more than 90 min from 2010 to 2019: 90.69%, HI
Largest % change decrease travel more than 90 min from 2010 to 2019: -18.76%, DC


In [37]:
#Calculating change in worked in state and change in worked in county over time (2010 to 2019) for each state

change_in_county = state_groupby['% Working in County of Residence'].pct_change().dropna() * 100 
change_out_county = state_groupby['% Working Outside County of Residence'].pct_change().dropna() * 100
change_in_state = state_groupby['% Working in State of Residence'].pct_change().dropna() * 100 
change_out_state = state_groupby['% Working Outside State of Residence'].pct_change().dropna() * 100


county_state_change = pd.DataFrame({'State': states, \
    '% Change Working in County of Residence': change_in_county,\
    '% Change Working Outside County of Residence': change_out_county, \
    '% Change Working in State of Residence': change_in_state,\
    '% Change Working Outside State of Residence': change_out_state})

county_state_change = county_state_change.reset_index(drop=True)

county_state_change.style.format({'% Change Working in County of Residence': '{:,.2f}%'.format, '% Change Working in State of Residence': '{:,.2f}%'.format,\
'% Change Working Outside County of Residence': '{:,.2f}%'.format, '% Change Working Outside State of Residence': '{:,.2f}%'.format})

county_state_change.to_csv('Resources/2010 to 2019 Travel County and State to Change.csv', index=False)

In [36]:
# Min and Max percentages - county state
max_in_county = county_state_change['% Change Working in County of Residence'].max()
min_in_county = county_state_change['% Change Working in County of Residence'].min()
max_in_state = county_state_change['% Change Working in State of Residence'].max()
min_in_state = county_state_change['% Change Working in State of Residence'].min()
max_out_county = county_state_change['% Change Working Outside County of Residence'].max()
min_out_county = county_state_change['% Change Working Outside County of Residence'].min()
max_out_state = county_state_change['% Change Working Outside State of Residence'].max()
min_out_state = county_state_change['% Change Working Outside State of Residence'].min()

# States corresponding to min and max
state_max_in_county = county_state_change[county_state_change['% Change Working in County of Residence'] == max_in_county]['State'].item()
state_min_in_county = county_state_change[county_state_change['% Change Working in County of Residence'] == min_in_county]['State'].item() 
state_max_in_state = county_state_change[county_state_change['% Change Working in State of Residence'] == max_in_state]['State'].item()
state_min_in_state = county_state_change[county_state_change['% Change Working in State of Residence'] == min_in_state]['State'].item()
state_max_out_county = county_state_change[county_state_change['% Change Working Outside County of Residence'] == max_out_county]['State'].item()
state_min_out_county = county_state_change[county_state_change['% Change Working Outside County of Residence'] == min_out_county]['State'].item()
state_max_out_state = county_state_change[county_state_change['% Change Working Outside State of Residence'] == max_out_state]['State'].item()
state_min_out_state = county_state_change[county_state_change['% Change Working Outside State of Residence'] == min_out_state]['State'].item()

print(f'Largest % change increase working in county of residence from 2010 to 2019: {round(max_in_county,2)}%, {state_max_in_county}')
print(f'Largest % change decrease working in county of residence from 2010 to 2019: {round(min_in_county,2)}%, {state_min_in_county}')
print(f'Largest % change increase working in state of residence from 2010 to 2019: {round(max_in_state,2)}%, {state_max_in_state}')
print(f'Largest % change decrease working in state of residence from 2010 to 2019: {round(min_in_state,2)}%, {state_min_in_state}')
print(f'Largest % change increase working ouside county of residence from 2010 to 2019: {round(max_out_county,2)}%, {state_max_out_county}')
print(f'Largest % change decrease working outside county of residence from 2010 to 2019: {round(min_out_county,2)}%, {state_min_out_county}')
print(f'Largest % change increase working outside state of residence from 2010 to 2019: {round(max_out_state,2)}%, {state_max_out_state}')
print(f'Largest % change decrease working outside state of residence from 2010 to 2019: {round(min_out_state,2)}%, {state_min_out_state}')

Largest % change increase working in county of residence from 2010 to 2019: 4.24%, MD
Largest % change decrease working in county of residence from 2010 to 2019: -6.25%, SD
Largest % change increase working in state of residence from 2010 to 2019: 2.21%, MD
Largest % change decrease working in state of residence from 2010 to 2019: -5.19%, RI
Largest % change increase working ouside county of residence from 2010 to 2019: 20.28%, HI
Largest % change decrease working outside county of residence from 2010 to 2019: -11.22%, WY
Largest % change increase working outside state of residence from 2010 to 2019: 85.9%, AK
Largest % change decrease working outside state of residence from 2010 to 2019: -28.88%, UT


#Time Traveled to Work Summary